### Imports

In [1]:
# Reuse previous LLM setup
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv

### Load API key & LLM

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1")

llm = ChatOpenAI(
    model="openai/gpt-oss-20b:free",
    temperature=0,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE,
)

In [3]:
def final_risk_score(case_data):
    """Calculate risk score based on suspicious patterns"""
    risk_score = (
        case_data.get('bank_shared_count', 0) * 0.4 +
        case_data.get('phone_shared_count', 0) * 0.3 +
        case_data.get('registrations_per_aadhaar', 0) * 0.2 +
        case_data.get('agent_cluster_size', 0) * 0.1
    )
    
    return {
        'risk_score': min(risk_score, 10),
        'risk_level': 'High' if risk_score > 7 else 'Medium' if risk_score > 4 else 'Low',
        'factors': [
            f"Bank shared count: {case_data.get('bank_shared_count', 0)}",
            f"Phone shared count: {case_data.get('phone_shared_count', 0)}",
            f"Registrations per Aadhaar: {case_data.get('registrations_per_aadhaar', 0)}"
        ]
    }

### Audit summary prompt template

In [4]:
audit_prompt = PromptTemplate(
    input_variables=["case_data", "risk_data", "explanation"],
    template="""
You are a senior government welfare fraud auditor.

Generate an official audit summary for the following suspicious beneficiary case.

CASE DATA:
{case_data}

RISK ANALYSIS:
{risk_data}

AI EXPLANATION:
{explanation}

Your report must include:

1. Case Overview
2. Key Fraud Indicators
3. Risk Score Interpretation
4. Evidence Summary
5. Recommended Government Action

Write in clear, formal, audit-ready language.
"""
)

audit_chain = LLMChain(
    llm=llm,
    prompt=audit_prompt
)


C:\Users\Dell\AppData\Local\Temp\ipykernel_27744\476703227.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  audit_chain = LLMChain(


In [ ]:
def generate_audit_summary(case_data, risk_data, explanation):
    """Generate audit summary using LLM"""
    prompt = f"""
You are a senior government welfare fraud auditor.

Generate an official audit summary for the following suspicious beneficiary case.

CASE DATA:
{case_data}

RISK ANALYSIS:
{risk_data}

AI EXPLANATION:
{explanation}

Your report must include:

1. Case Overview
2. Key Fraud Indicators
3. Risk Score Interpretation
4. Evidence Summary
5. Recommended Government Action

Write in clear, formal, audit-ready language.
"""
    
    try:
        response = llm.invoke([{"role": "user", "content": prompt}])
        return response.content
    except Exception as e:
        return f"Error generating audit summary: {str(e)}"

### Run full pipeline (Explanation → Risk → Audit)

In [6]:
# Example suspicious case
suspicious_case = {
    "aadhaar_like_id": "4587 9214 6632",
    "phone_number": "9876543210",
    "bank_account": "123456789012",
    "registrations_per_aadhaar": 3,
    "bank_shared_count": 5,
    "phone_shared_count": 4,
    "agent_cluster_size": 18,
    "annual_income": "₹85,000",
    "district": "Bhopal"
}

# --- Step 1: Explanation (from previous LangGraph result) ---
explanation_text = "Mock explanation: This beneficiary shows multiple red flags including high shared banking relationships and multiple registrations per Aadhaar ID, indicating potential fraudulent activity in the welfare program."# reuse earlier output

# --- Step 2: Risk score (from previous module) ---
risk_output = final_risk_score(suspicious_case)

# --- Step 3: Audit summary ---
audit_report = generate_audit_summary(
    suspicious_case,
    risk_output,
    explanation_text
)

print("\n🧾 FINAL AUDIT REPORT\n")
print(audit_report)



🧾 FINAL AUDIT REPORT

Error generating audit summary: name 'client' is not defined
